In [1]:
import pandas as pd

## Register Status

In [2]:
def ini_register_status():
    register_status = {}

    # Registradores inteiros (x0..x31)
    for i in range(32):
        register_status[f"x{i}"] = {"x": i, "x_type": "int", "writer": None}

    # Registradores de ponto flutuante (f0..f31)
    for i in range(32):
        register_status[f"f{i}"] = {"f": i, "f_type": "float", "writer": None}
    return register_status


## Parser das instruções

In [3]:
OPCODES = {
    'fld': 0,
    'fsd': 1,
    'fadd': 2,
    'fsub': 3,
    'fmul': 4,
    'fdiv': 5
}

# Define register prefix constants
REG_PREFIXES = {
    'x': 'int',
    'f': 'float'
}

In [4]:
def init_instruction_status(program_name):
    instruction_status =[]
    with open(program_name, "r") as file:
        for i, line in enumerate(file):
            op = OPCODES[line.split()[0]]
            if (op == 0 or op==1):
                rd = line.split()[1].replace(",", "")
                rs1 = line.split()[2].split("(")[1].replace(")", "")
                rs2 = None
                imm = line.split()[2].split("(")[0]
                fu_type = 0
            else:
                if op == 2 or op == 3: fu_type = 2
                elif op == 4: fu_type = 1
                elif op == 5: fu_type = 3
                rd = line.split()[1].replace(",", "")
                rs1 = line.split()[2].replace(",", "")
                rs2 = line.split()[3].replace(",", "")
                imm = None          
            instruction_status.append({ 
                "id": i,
                "inst": line.replace("\n", ""),
                "opcode": op,
                "fu_name": None,
                "fu_type": fu_type,
                "rd": rd,
                "rs1": rs1,
                "rs2": rs2,
                "imm": imm,
                "issue": None,
                "read_operands": None,
                "execution_complete": None,
                "write_result": None

            })
    return  instruction_status

## Parser das functionals units

In [5]:
def parser_fus_configs(config_name):
    fus_configs = {}
    with open(config_name, 'r') as f:
        linhas = f.readlines()
        for linha in linhas:
            partes = linha.strip().split()
            fus_configs[partes[0]] = {'qtd': partes[1], 'cycles': partes[2]}
    return fus_configs

In [6]:
def init_fus_status(fus_configs):
    fus_configs = parser_fus_configs(config_name="tests/uf_config1.in")
    fus_status = {}
    for uf_type, config in fus_configs.items():
        if uf_type == 'int':
            op = 0
        elif uf_type == 'mult':
            op = 1
        elif uf_type == 'add':
            op = 2
        elif uf_type == 'div':
            op = 3
        for c in range(int(config['qtd'])):
            fus_status[f'{uf_type}{c+1}'] = {
                'Opcode': op, 
                'Busy': False, 
                'Op': None, 
                'Fi': None,
                'Fj': None, 
                'Fk': None, 
                'Qj': None, 
                'Qk': None,
                'Rj': False, 
                'Rk': False, 
                'Cycles_left': int(config['cycles']), 
                'Cycles': int(config['cycles'])
            }
    return fus_status

## ISSUE

Verificar se a unidade funcional requerida está livre -- sem hazards estruturais
    
    Functional unit status -- verificada aqui

Verificar se nenhuma outra instrução vai escrever no rd -- sem WAW hazards
    
    Register Status -- verificado aqui

In [7]:
def issue(instr, fus_status, register_status):
    opcode_i = instr['opcode']
    id_i = instr['id']
    rs1_i = instr['rs1']
    rs2_i = instr['rs2']
    rd_i = instr['rd']
    fu_type = instr['fu_type']

    print(f"\n[ISSUE] Tentando emitir instrução {id_i}: {instr['inst']}")

    for fu_name, fu in fus_status.items():
        print(f"  Checando FU {fu_name} (Opcode esperado={fu_type}, FU Opcode={fu['Opcode']}, Busy={fu['Busy']})")

        # 1. opcode não bate
        if fu['Opcode'] != fu_type:
            print(f"    -> Falha: FU {fu_name} não suporta opcode {fu_type}")
            continue

        # 2. FU já ocupada
        if fu['Busy']:
            print(f"    -> Falha: FU {fu_name} ocupada (Busy=True)")
            continue

        # 3. registrador destino já tem writer
        if register_status[rd_i]['writer'] is not None:
            print(f"    -> Falha: registrador {rd_i} já tem writer {register_status[rd_i]['writer']}")
            return False

        # Se passou em tudo, ocupa FU
        print(f"    -> Sucesso: Instr {id_i} emitida na FU {fu_name}")
        fu['Busy'] = True
        fu['Op'] = id_i
        fu['Fi'] = rd_i
        fu['Fj'] = rs1_i
        fu['Fk'] = rs2_i
        fu['Qj'] = register_status[rs1_i]['writer'] if rs1_i is not None else None
        fu['Qk'] = register_status[rs2_i]['writer'] if rs2_i is not None else None
        fu['Rj'] = fu['Qj'] is None
        fu['Rk'] = fu['Qk'] is None
        register_status[rd_i]['writer'] = fu_name
        instr['fu_name'] = fu_name
        return True

    print(f"  -> Nenhuma FU disponível para emitir instrução {id_i}")
    return False


## Read Operands

verifica no `register_status` se alguma unidade funcional vai escrever nos operandos de leitura `rs1` e `rs2`

In [ ]:
def read_operands(fu, inst, register_status, instruction_status, cycle):
            
            print(f"[DEBUG] Estado inicial da FU {inst['fu_name']}: {fu}")

            if not fu['Rj']:
                writer_j = register_status.get(fu['Fj'], {}).get('writer')
                print(f"[DEBUG] FU={inst['fu_name']} -> Registrador Fj={fu['Fj']} | writer_j={writer_j}")

                if writer_j is None:  # ninguém está escrevendo em Rj
                    fu['Rj'] = True
                    fu['Qj'] = None
                    print(f"[DEBUG] Fj livre -> FU['Rj'] = True, Qj=None")
                else:
                    writer_inst = next((i for i in instruction_status if i['fu_name'] == writer_j), None)
                    print(f"[DEBUG] Dependência encontrada: writer_inst={writer_inst}")
                    if writer_inst and writer_inst['write_result'] is not None and inst['write_result'] < cycle: # somente se o write estiver completo
                        fu['Rj'] = True
                        fu['Qj'] = None
                        print(f"[DEBUG] Escritor já escreveu os resultados -> FU['Rj'] = True, Qj=None")

            # --- Checando Rk ---
            if not fu['Rk']:
                print(f"[DEBUG] FU={inst['fu_name']} -> Registrador Fk={fu['Fk']}")
                if not fu['Fk']:  # não tem Fk
                    fu['Rk'] = True
                    print(f"[DEBUG] Fk inexistente -> FU['Rk'] = True")
                else:
                    writer_k = register_status.get(fu['Fk'], {}).get('writer')
                    print(f"[DEBUG] FU={inst['fu_name']} -> writer_k={writer_k}")
                    if writer_k is None:
                        fu['Rk'] = True
                        fu['Qk'] = None
                        print(f"[DEBUG] Fk livre -> FU['Rk'] = True, Qk=None")

            # --- Se pode ler operandos ---
            if fu['Rj'] and fu['Rk']:
                inst['read_operands'] = cycle
                fu['Rj'] = False
                fu['Rk'] = False
                print(f"[SUCCESS] Ciclo {cycle}: Instrução '{inst['inst']}' leu os operandos.")
                print(f"[DEBUG] Estado final da FU {inst['fu_name']}: {fu}")

## Complete Execution

In [9]:

def execute(fu, cycle, inst):
    if fu['Cycles_left'] > 0:
        fu['Cycles_left'] -= 1
        print(f"INFO: Instrução '{inst['inst']}' está executando. Ciclos restantes: {fu['Cycles_left']}.")
    if fu['Cycles_left'] == 0:
        inst['execution_complete'] = cycle
        fu['Cycles_left'] = fu['Cycles']
        print(f"OK: Instrução '{inst['inst']}' completou a execução.")

## Write Results

In [10]:
def write_results(fus_status, inst, cycle, register_status):
    fu = fus_status[inst['fu_name']]
    fi_register = fu['Fi'] # O registrador de destino desta instrução

    # --- VERIFICAÇÃO DE HAZARD WAR ---
    # Itera sobre todas as unidades funcionais para ver se alguma precisa ler nosso registrador de destino.
    for other_fu_name, other_fu in fus_status.items():
        # Ignora a verificação contra si mesma
        if other_fu_name == inst['fu_name']:
            continue

        # Condição de Hazard: Outra FU ainda não leu (Rj=False) e seu operando fonte (Fj)
        # é o nosso registrador de destino (Fi).
        if other_fu['Rj'] and other_fu['Fj'] == fi_register:
            print(f"STALL (WAR): Write Back de '{inst['inst']}' pausado. FU '{other_fu_name}' ainda precisa ler {fi_register}.")
            return # Stall! Não faz nada neste ciclo.

        # Repete a verificação para o segundo operando (Fk)
        if other_fu['Rk'] and other_fu['Fk'] == fi_register:
            print(f"STALL (WAR): Write Back de '{inst['inst']}' pausado. FU '{other_fu_name}' ainda precisa ler {fi_register}.")
            return # Stall!True

    # --- Se passou na verificação (sem hazards), procede com o Write Back ---
    inst['write_result'] = cycle
    print(f"OK: Instrução '{inst['inst']}' fez Write Back.")

    if fi_register and register_status[fi_register]['writer'] == inst['fu_name']:
        register_status[fi_register]['writer'] = None
        print(f"Liberando registrador de destino {fi_register}")

    fu.update({
        'Busy': False, 'Op': None, 'Fi': None, 'Fj': None, 'Fk': None,
        'Qj': None, 'Qk': None, 'Rj': False, 'Rk': False
    })

## Scoreboard

### Montando a instrução novamente

In [20]:
prog_file = "tests/ex2.s"
fus_file = "tests/uf_config2.in"

In [12]:
def has_pending_instructions(inst_status):
    return any(inst['write_result'] is None for inst in inst_status)


fld f1, 0(x1)
fld f5, 0(x1)

In [13]:
register_status = ini_register_status()
instruction_status = init_instruction_status(prog_file)
fus_status = init_fus_status(fus_file)

Debug

In [21]:
register_status = ini_register_status()
instruction_status = init_instruction_status(prog_file)
fus_status = init_fus_status(fus_file)
pc = 0
cycle = 1

while pc < len(instruction_status) or has_pending_instructions(instruction_status):
    print(f"\n--- Cycle {cycle} ---")
    print(f"PC = {pc}")
    liberar_write = False
    # WRITE RESULTS
    for inst in instruction_status:
        if inst['execution_complete'] is not None and inst['execution_complete'] < cycle and inst['write_result'] is None:
            write_results(fus_status, inst, cycle, register_status)
            liberar_write = True
            
    for inst in instruction_status:
        if inst['read_operands'] is not None and inst['read_operands'] < cycle and inst['execution_complete'] is None:
            fu = fus_status[inst['fu_name']]
            execute(fu, cycle, inst)

    # 1. Read operands
    for inst in instruction_status:
        if inst['issue'] is not None and inst['read_operands'] is None:
            if liberar_write:
                print(f"[DEBUG] Ciclo {cycle}: leitura bloqueada porque houve write")
                continue  # pula leitura neste ciclo
            
            print(f"\n[DEBUG] Ciclo {cycle} - Checando instrução '{inst['inst']}' (FU={inst['fu_name']})")

            fu = fus_status[inst['fu_name']]
            read_operands(fu, inst, register_status, instruction_status, cycle)

    # 2. Issue
    if pc < len(instruction_status):
        if not liberar_write:
            next_instruction = instruction_status[pc]
            if issue(next_instruction, fus_status, register_status):
                next_instruction['issue'] = cycle
                print(f"--> Instr {next_instruction['id']} emitida")
                pc += 1
            else:
                print(f"--> Instr {next_instruction['id']} NÃO pôde ser emitida")
                # print(issue(next_instruction, fus_status, register_status))

    cycle += 1



--- Cycle 1 ---
PC = 0

[ISSUE] Tentando emitir instrução 0: fld  f1, 100(x7)
  Checando FU int1 (Opcode esperado=0, FU Opcode=0, Busy=False)
    -> Sucesso: Instr 0 emitida na FU int1
--> Instr 0 emitida

--- Cycle 2 ---
PC = 1

[DEBUG] Ciclo 2 - Checando instrução 'fld  f1, 100(x7)' (FU=int1)
[DEBUG] Estado inicial da FU int1: {'Opcode': 0, 'Busy': True, 'Op': 0, 'Fi': 'f1', 'Fj': 'x7', 'Fk': None, 'Qj': None, 'Qk': None, 'Rj': True, 'Rk': True, 'Cycles_left': 1, 'Cycles': 1}
[SUCCESS] Ciclo 2: Instrução 'fld  f1, 100(x7)' leu os operandos.
[DEBUG] Estado final da FU int1: {'Opcode': 0, 'Busy': True, 'Op': 0, 'Fi': 'f1', 'Fj': 'x7', 'Fk': None, 'Qj': None, 'Qk': None, 'Rj': False, 'Rk': False, 'Cycles_left': 1, 'Cycles': 1}

[ISSUE] Tentando emitir instrução 1: fmul f2, f2, f4
  Checando FU int1 (Opcode esperado=1, FU Opcode=0, Busy=True)
    -> Falha: FU int1 não suporta opcode 1
  Checando FU int2 (Opcode esperado=1, FU Opcode=0, Busy=False)
    -> Falha: FU int2 não suporta opcod

## Print Resultado

In [15]:
import pandas as pd

def formatar_tabela_scoreboard(instruction_status):
    df = pd.DataFrame(instruction_status)
    tabela_final = df[['inst', 'issue', 'read_operands', 'execution_complete', 'write_result']]

    tabela_final = tabela_final.rename(columns={
        'inst': 'Instruction/Cicle',
        'issue': 'Issue',
        'read_operands': 'Read',
        'execution_complete': 'Execute', 
        'write_result': 'Write'
    })

    tabela_final = tabela_final.fillna("-")

    return tabela_final

In [22]:
tabela_de_resultados = formatar_tabela_scoreboard(instruction_status)
print(tabela_de_resultados.to_string(index=False))

Instruction/Cicle  Issue  Read  Execute  Write
 fld  f1, 100(x7)      1     2        3      4
  fmul f2, f2, f4      2     3        7      8
  fadd f2, f1, f3      9    10       12     13
   fld  f9, 0(x3)     10    11       12     13
  fdiv f3, f1, f7     11    12       22     23
  fsub f6, f3, f4     14    24       26     27
  fmul f7, f1, f2     15    16       20     21
  fadd f4, f5, f2     28    29       31     32
 fsd  f1, 50(x11)     29    30       31     32
